In [0]:
import numpy as np
import pandas as pd
import time
import math
import random

np.random.seed(2)

ACTIONS = ['lock', 'open']

# Simulation Length: Refresh Every 7 days, Run for 52 weeks
MAX_RUN = 52

# Greedy Policy Rate
EPSILON = 0.8

# Learning Rate
ALPHA = 0.1

# Decay Rate
GAMMA = 0.9

MAX_EPISODES = 500
FRESH_TIME = 0.1

## Epidemic Development Model

In [0]:
def create_citizen_array(width, height):
  citizen = np.zeros((width, height), dtype=int)
  print(citizen)
  return citizen

In [0]:
def random_assign_patient(total, citizen):
  size = citizen.shape
  patient_list = []

  for i in range(total):
    x = np.random.randint(0, size[0], 1)[0]
    y = np.random.randint(0, size[1], 1)[0]
    citizen[x][y] = R0
    patient_list.append([x, y])
  print(citizen)
  print(patient_list)
  return patient_list, citizen

In [0]:
def infectNeighbour(patient, citizen):
  x = patient[0]
  y = patient[1]

  width = citizen.shape[0]
  height = citizen.shape[1]

  for i in range(-1, 1):
    for j in range(-1, 1):
      if ((i+j) != 0):
        x_ = x + i
        y_ = y + j
        if (0 <= x_) and (x_ < width) and (0 <= y_) and (y_ <= height) and (citizen[x_][y_] == 0):
          return x_, y_
  return -1, -1

In [0]:
def spread_virus(cycle, R0, lockFactor, patient_list, citizen):
  size = citizen.shape
  width = size[0]
  height = size[1]

  patient_list_copy = patient_list.copy()
  remove_count = 0

  '''
  for patient in patient_list_copy:
    for i in range(R0):
      direc = np.random.randint(-1, 2, 2)
      newX = patient[0]+direc[0]
      newY = patient[1]+direc[1]
      if (newX >= 0) and (newX < width) and (newY >= 0) and (newY < height) and (citizen[newX][newY] == 0):
        citizen[newX][newY] = cycle;
        patient_list.append([newX, newY])
  return patient_list, citizen
  '''

  for patient in patient_list_copy:
    # Patient still being infectious
    if (citizen[patient[0]][patient[1]] > 0):
      citizen[patient[0]][patient[1]] -= 1
      if (np.random.uniform() <= lockFactor):  # If not lock, randomly infect, otherwise definitely infect
        x, y = infectNeighbour(patient, citizen)
        if ((x+y) >= 0): 
          citizen[x][y] = R0
          patient_list.append([x, y])
    else: # R0 reached, mark this patient as recoverd
      patient_list.remove(patient)
      remove_count+=1
  return patient_list, citizen

In [0]:
def patient_recover(patient_list, citizen):
  patient_list_copy = patient_list.copy()
  for patient in patient_list_copy:
    if (np.random.uniform() <= 0.3):
      citizen[patient[0]][patient[1]] = 0
      patient_list.remove(patient)
  return patient_list, citizen

In [0]:
def update_lock_effect(action, previous_action, lock_effect):
  if (action == 'lock') and (previous_action == 'lock'):
    lock_effect -= 0.05
  elif (action == 'open') and (previous_action == 'lock'):
    lock_effect += 0.05
  elif (action == 'open') and (previous_action == 'open'):
    lock_effect += 0.1
  else:
    lock_effect = 0.3

  if (lock_effect < 0.03):
    lock_effect = 0.05
  elif (lock_effect > 0.3):
    lock_effect = 0.3
  return  lock_effect

In [0]:
def update_u_rate(delta_infect, action, u):
  delta_infect = delta_infect / POPULATION
  if action == "lock":
    z = 1.05
  else:
    z = 0.975
  u_ = u * (1+math.sqrt(delta_infect)) * z
  if u_ > 1:
    u_ = 1
  elif (u_< 0.035):
    u_ = 0.035
  return u_

## Q Learning Utilities

In [0]:
# build a q table with size (population x 2)
def build_q_table(population, actions):
  table = pd.DataFrame(
      np.zeros((population, len(actions))), 
      columns = ACTIONS
  )
  print(table)
  return table

In [0]:
# Check Q-table
def choose_action(infected, q_table):
  state_actions = q_table.iloc[infected, :]

  if (np.random.uniform() > EPSILON):
    action_name = np.random.choice(ACTIONS)
  else:   # act greedy
    action_name = state_actions.idxmax()
  return action_name

In [0]:
# Environment
# Give a state, return a calculated reward
def get_env_feedback(patient_list, citizen, unemploy_rate, step, action, lock_effect):
  pl = patient_list
  c = citizen

  if (action == 'open'):
    pl, c = spread_virus(step, R0, 1, pl, c)
  else: # (action == 'lock'):
    pl, c = spread_virus(step, R0, lock_effect, pl, c)

  R = (1-unemploy_rate)*100 - (len(patient_list) / POPULATION)*10
  return pl, c, R

In [0]:
def calculate_terminal_reward(patient_list, unemployment, step_counter):
  if (len(patient_list) <= 0):
    R = (52-step_counter) - (len(patient_list) / POPULATION)*10 + (1-unemployment)*100
  elif (step_counter < 52):
    R = (1-unemployment)*100 - step_counter - (len(patient_list) / POPULATION)*10
  else:
    R = (1-unemployment)*100 - (len(patient_list) / POPULATION)*10

  print(step_counter, "final:", R)
  return R

In [0]:
def update_env(INFECTED, STEP, U, ACTION, REWARD):
    print("Week %d, Action: %s, %d / %.1f, Reward %.2f" % (STEP, ACTION, INFECTED, U*100, REWARD))
    time.sleep(FRESH_TIME)

## Q-Learning

In [0]:
WIDTH = 1000
HEIGHT = 1000
POPULATION = WIDTH*HEIGHT
Patient_Zeros = 100

In [0]:
# TD Solution
def rl():
  q_table = build_q_table(int(POPULATION / 100), ACTIONS)
  pl, c = random_assign_patient(Patient_Zeros, create_citizen_array(WIDTH,HEIGHT))

  best_case = 0
  best_infect = []
  best_unemploy = []

  for episode in range(MAX_EPISODES):
    step_counter = 1
    lock_effect = Lock_Effect
    U = Base_Unemployment_Rate

    patient_list = pl.copy()
    citizen = c.copy()

    S = int(Patient_Zeros / 100)
    is_terminated = False
    Inf = Patient_Zeros

    pA = 'open'

    infect_log = []
    unemploy_log = []
    final_reward = 0

    print("------------------------------")
    print("#", episode)
    print("------------------------------")
    
    while not is_terminated:
      # Choose an Action
      A = choose_action(Patient_Zeros, q_table) 
      
      # Interact with Enviroment
      lock_effect = update_lock_effect(A, pA, lock_effect)
      U = update_u_rate((len(patient_list)-Inf)/POPULATION, A, U)
      patient_list, citizen, R = get_env_feedback(patient_list, citizen, U, step_counter, A, lock_effect)

      # Get next state
      Inf = len(patient_list)
      S_ = int(Inf / 100)

      q_predict = q_table.loc[S, A]
      if (Inf >= (POPULATION*0.02)) or (step_counter > MAX_RUN) or (Inf == 0):
        q_target = calculate_terminal_reward(patient_list, U, step_counter)
        is_terminated = True
        final_reward = q_target
      else:
        q_target = R + GAMMA * q_table.iloc[S_, :].max() # Check diff between Real Value & Estimated Value
      
      q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # Update Q-table
      
      S = S_
      pA = A
      update_env(Inf, step_counter, U, A)
      infect_log.append(Inf)
      unemploy_log.append(U)
      step_counter += 1
    if (final_reward > best_case):
      best_infect = list(infect_log)
      best_unemploy = list(unemploy_log)
      best_case = final_reward
    print(q_table)
  return q_table, best_case, best_infect, best_unemploy

In [0]:
q, best_case, best_infect, best_unemploy = rl()
print(q)
print(best_case)
print(best_infect)
print(best_unemploy)

## Deep Q Network

In [0]:
import torch
import torch.nn as nn
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
"""
Reinforcement learning maze example.
Red rectangle:          explorer.
Black rectangles:       hells       [reward = -1].
Yellow bin circle:      paradise    [reward = +1].
All other states:       ground      [reward = 0].
This script is the environment part of this example.
The RL is in RL_brain.py.
View more on my tutorial page: https://morvanzhou.github.io/tutorials/
"""

R0 = 4
Lock_Effect = 0.5
Base_Unemployment_Rate = 0.035
MAX_RUN = 52
FRESH_TIME = 0.1
POPULATION = 1000*1000

class Environment(object):
    def __init__(self):
        super(Environment, self).__init__()
        self.action_space = ['lock', 'open', 'mask', 'togo']
        self.n_actions = len(self.action_space)
        self.state = None
        self.u = None
        self.pl = None
        self.citizen = None
        self.le = Lock_Effect
        self.last_action = None

    def reset(self):
        self.citizen = create_citizen_array(1000, 1000)
        self.pl, self.citizen = random_assign_patient(100, self.citizen)
        self.u = 3.5 / 100
        self.state = (len(self.pl), self.u)
        return np.array(self.state)
    
    def instant_reward(self, infect, step):
      infect_rate = infect / POPULATION * 100 # 0-2
      unemployment_rate = self.u * 100
      r1 = 1 - infect_rate # [-1, 1)
      r2 = 7.5 - unemployment_rate  #[-10, 4]
      return (r1*0.5 + r2 / 4 * 0.3 + (step / 51) * 0.2)
    
    def terminate_reward(self, infect, step):
      ri = self.instant_reward(infect)
      R = step / 51 * 1.5
      return (R+ri)

    def update_u(self, delta_infect, action, u):
      delta_infect = delta_infect / POPULATION
      if action == "lock":
        z = 1.05
      elif action == "open":
        z = 0.975
      else:
        z = 1.01

      if (delta_infect < 0):
        u_ = u * z
      else:
        u_ = u * (1+math.sqrt(delta_infect)) * z
      if u_ > 1:
        u_ = 1
      elif (u_< 0.035):
        u_ = 0.035
      return u_

    def step(self, action, step):
      self.le = update_lock_effect(action,  self.last_action, self.le)
      last_infect = len(self.pl)

      if (self.action_space[action] == 'open'):
        self.pl, self.citizen = spread_virus(step, R0, 1, self.pl, self.citizen)
      elif (self.action_space[action] == 'mask'):
        self.pl, self.citizen = spread_virus(step, R0-2, 1, self.pl, self.citizen)
      elif (self.action_space[action] == 'togo'):
        self.pl, self.citizen = spread_virus(step, R0, 0.8, self.pl, self.citizen)
      else: # (action == 'lock'):
        self.pl, self.citizen = spread_virus(step, R0, self.le, self.pl, self.citizen)

      infect = len(self.pl)
      delta_infect = infect - last_infect
      self.u = self.update_u(delta_infect, self.action_space[action], self.u)

      self.state = (infect, self.u)
      reward = self.instant_reward(infect, step)

      if (step >= 51) or (infect >= 0.02*POPULATION) or (self.u >= 0.16):
        done = True
        #reward = self.terminate_reward(infect, step)
        #print("tr:", reward)
      else:
        done = False
      
      update_env(infect, step, self.u, self.action_space[action], reward)
      time.sleep(0.02)
      return np.array(self.state), reward, done

In [0]:
class DeepQNetwork(nn.Module):
  def __init__(self):
    super(DeepQNetwork, self).__init__()
    self.fc = nn.Sequential(
        nn.Linear(2, 24),
        nn.ReLU(),
        nn.Linear(24, 24),
        nn.ReLU(),
        nn.Linear(24, 4)
    )
    self.mls = nn.MSELoss()
    self.opt = torch.optim.Adam(self.parameters(), lr=0.01)
  
  def forward(self, inputs):
    return self.fc(inputs)

In [0]:
memory_size = 400
decline = 0.6
update_time = 50
gama = 0.9
b_size = 200
memory = np.zeros((memory_size, 6)) # S(2)   A(1)   S_(2)   R(1)

def run_deepQ(env, net, net2):
  memory_count = 0
  learn_time = 0

  for episode in range(300):
    obervation = env.reset()
    step = 0

    while True:
      if (learn_time < 10):
        if (memory_count < memory_size):
          print(learn_time, memory_count)
        print(learn_time)

      if random.randint(0, 100) < 100 * (decline ** learn_time):
        action = random.randint(0, 3)  # Randomly pick an action
      else:
        out = net(torch.Tensor(obervation)).detach()  # [left_rewards_total, right_reward_total]
        action = torch.argmax(out).data.item()

      observation_, reward, done = env.step(action, step)
      
      observation = observation_ 

      idx = memory_count % memory_size
      memory[idx][0:2] = observation
      memory[idx][2:3] = action
      memory[idx][3:5] = observation_
      memory[idx][5:6] = reward
      memory_count +=1

      if (memory_count >= memory_size):  # Start to learn
        learn_time += 1 # Learn once
        if (learn_time % update_time == 0): # Sync two nets
          net2.load_state_dict(net.state_dict())
          print("Sync Two Net")
        else:
          rdp = random.randint(0, memory_size - b_size - 1)
          b_s = torch.Tensor(memory[rdp:rdp+b_size, 0:2])
          b_a = torch.Tensor(memory[rdp:rdp+b_size, 2:3]).long()
          b_s_ = torch.Tensor(memory[rdp:rdp+b_size, 3:5])
          b_r = torch.Tensor(memory[rdp:rdp+b_size, 5:6])

          q = net(b_s).gather(1, b_a)
          q_next = net2(b_s_).detach().max(1)[0].reshape(b_size, 1)
          tq = b_r + gama * q_next
          loss = net.mls(q, tq)
          net.opt.zero_grad()
          loss.backward()
          net.opt.step()

      step +=1
      if done:
        break

In [0]:
net = DeepQNetwork()
net2 = DeepQNetwork()
env = Environment()

run_deepQ(env, net, net2)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[826, 595], [725, 47], [29, 916], [876, 823], [442, 301], [452, 151], [674, 463], [647, 697], [786, 921], [335, 352], [957, 519], [481, 425], [403, 160], [949, 0], [855, 730], [639, 39], [84, 518], [749, 3], [731, 276], [969, 979], [948, 363], [931, 294], [158, 357], [871, 946], [341, 336], [643, 837], [742, 690], [705, 968], [129, 954], [881, 282], [981, 581], [838, 610], [482, 309], [701, 843], [118, 239], [586, 200], [773, 6], [679, 111], [94, 145], [663, 544], [451, 41], [109, 521], [103, 541], [879, 273], [699, 893], [759, 659], [711, 112], [750, 353], [7, 643], [183, 959], [331, 479], [830, 744], [767, 744], [462, 741], [810, 333], [705, 821], [387, 253], [710, 324], [51, 526], [550, 706], [243, 510], [896, 940], [873, 603], [54, 628], [487, 9

KeyboardInterrupt: ignored